In [ ]:
import pandas as pd
import glob
from geopandas import GeoDataFrame
import geopandas as gpd
from math import sin, cos, sqrt, atan2, radians
import numpy as np
import matplotlib.pyplot as plt
import dateutil 
import datetime as dt
from datetime import datetime, timedelta
%matplotlib inline
plt.style.use('ggplot')

# approximate radius of earth in km
R = 6373.0

# Creating first df

In [ ]:
df = pd.read_csv('serverdata/2017-09-07 23:00:022000obikesZH.csv')

In [ ]:
del df['Unnamed: 0']
del df['index']
del df['countyId']
del df['helmet']
del df['imei']

In [ ]:
def split1(elem):
    elem = elem.replace('POINT (', '')
    elem = elem.replace(')', '')
    return elem.split(' ')[0]

In [ ]:
def split2(elem):
    elem = elem.replace('POINT (', '')
    elem = elem.replace(')', '')
    return elem.split(' ')[1]

In [ ]:
df['Long'] = df['2017-09-07 23:00:02'].apply(split1)
df['Lat'] = df['2017-09-07 23:00:02'].apply(split2)
df['Timestamp'] = '2017-09-07 23:00:02'
del df['2017-09-07 23:00:02']

In [ ]:
df.head(3)

# Creating list of names in file

In [ ]:
file_names = []
for name in glob.glob('serverdata/*'):
    name = name.split('/')[-1]
    file_names.append(name)

# Iterating through contents of all the server files

In [ ]:
for file in file_names[0:]:
    df_new = pd.read_csv('serverdata/'+ file)
    
    del df_new['Unnamed: 0']
    del df_new['index']
    del df_new['countyId']
    del df_new['helmet']
    del df_new['imei']
    
    df_new['Long'] = df_new[file[:19]].apply(split1)
    df_new['Lat'] = df_new[file[:19]].apply(split2)
    df_new['Timestamp'] = file[:19]
    del df_new[file[:19]]
    #print(file[:19])
    frames = [df, df_new]

    df = pd.concat(frames)

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.to_csv('timestamps.csv')

# Die beliebtesten Standorte

In [ ]:
df['Long'].value_counts(ascending=False).head(5)

In [ ]:
df['Lat'].value_counts(ascending=False).head(5)

# How many bikes are there in total?

In [ ]:
id_list = list(set(list(df['id'])))
len(id_list)

# Counting which bikes have the most location changes?

In [ ]:
id_list = list(set(list(df['id'])))
list_count = []
for elem in id_list:
    count = df[df['id'] == elem]['Long'].value_counts().count()
    mini_dict = {'Count':count,
                 'ID': elem}
    list_count.append(mini_dict)
    df_change_count = pd.DataFrame(list_count)

In [ ]:
print("Startzeit:", df['Timestamp'].head(1)[0],
     "Endzeit:",str(df['Timestamp'].tail(1))[:26][7:])

In [ ]:
df_change_count.sort_values(by='Count', ascending=False).head(10)

# Average amount of time bikes are used in the above defined time period?

In [ ]:
df_change_count['Count'].mean()

# How many bikes never moved in the past three days?

In [ ]:
df_change_count[df_change_count['Count'] == 1].count()

# Percentage of total? (Bikes that haven't moved in the above mentioned time frame)

In [ ]:
total = len(list(set(list(df['id']))))

In [ ]:
never_moved = df_change_count[df_change_count['Count'] == 1]['Count'].count()

In [ ]:
round(never_moved / total * 100, 1)

# Save list of bikes that have never moved, and created a del list

In [ ]:
Del_list = list(df_change_count[df_change_count['Count'] == 1]['ID'])

# Creating DataFrame of the bikes that have never moved, with location data

In [ ]:
Del_list[0:5]

In [ ]:
df_never_moved = df.head(0)

In [ ]:
for id_never_moved in Del_list:
    df_new = df[df['id'] == id_never_moved]
    frames = [df_never_moved, df_new]
    df_never_moved = pd.concat(frames)

In [ ]:
df_never_moved = df_never_moved.drop_duplicates(subset='id')

In [ ]:
df_never_moved.info() #remove Time

In [ ]:
df_never_moved.to_csv('never_moved.csv')

In [ ]:
df_never_moved['Long'].value_counts().head(5)

In [ ]:
df_never_moved['Lat'].value_counts().head(10)

# Delete these bikes from the list

In [ ]:
for bike in Del_list:
    id_list.remove(bike)

In [ ]:
len(id_list)

# Create list of bikes that have moved

In [ ]:
rel_list_count = []
for elem in id_list:
    count = df[df['id'] == elem]['Long'].value_counts().count()
    mini_dict = {'Count':count,
                 'ID': elem}
    rel_list_count.append(mini_dict)

In [ ]:
df_moved = pd.DataFrame(rel_list_count)

In [ ]:
df_moved.sort_values(by='Count', ascending=False).head()

# The bike with the most movements

In [ ]:
df[df['id']==41000425].to_csv('BikeID41000425.csv')

# What is the average distance per bike move?

# First the top bike

In [ ]:
top_bike = df[df['id']==41000425]

In [ ]:
long_list = list(top_bike['Long'])
last_elem = long_list[-1]
long_list.append(last_elem)
long_list.pop(0)
top_bike['newLong'] = long_list

In [ ]:
lat_list = list(top_bike['Lat'])
last_elem = lat_list[-1]
lat_list.append(last_elem)
lat_list.pop(0)
top_bike['newLat'] = lat_list

In [ ]:
la1 = list(top_bike['Lat'])
lo1 = list(top_bike['Long'])
la2 = list(top_bike['newLat'])
lo2 = list(top_bike['newLong'])

In [ ]:
distance_list = []

for lat1, lon1, lat2, lon2 in zip(la1, lo1, la2, lo2):

    lat1 = radians(float(lat1))
    lon1 = radians(float(lon1))
    lat2 = radians(float(lat2))
    lon2 = radians(float(lon2))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    distance_list.append(distance)

In [ ]:
top_bike['Distance'] = distance_list

In [ ]:
final_topbike = top_bike[top_bike['Distance'] != 0.000000]

In [ ]:
final_topbike.head(0)

# Now with all the bikes

In [ ]:
final_topbikes = final_topbike.head(0)

In [ ]:
moved_bikes = list(df_moved['ID'])

In [ ]:
for bike in moved_bikes:
    
    top_bike = df[df['id']== bike]
    
    long_list = list(top_bike['Long'])
    last_elem = long_list[-1]
    long_list.append(last_elem)
    long_list.pop(0)
    top_bike['newLong'] = long_list
    
    lat_list = list(top_bike['Lat'])
    last_elem = lat_list[-1]
    lat_list.append(last_elem)
    lat_list.pop(0)
    top_bike['newLat'] = lat_list
    
    la1 = list(top_bike['Lat'])
    lo1 = list(top_bike['Long'])
    la2 = list(top_bike['newLat'])
    lo2 = list(top_bike['newLong'])
    
    distance_list = []

    for lat1, lon1, lat2, lon2 in zip(la1, lo1, la2, lo2):
        lat1 = radians(float(lat1))
        lon1 = radians(float(lon1))
        lat2 = radians(float(lat2))
        lon2 = radians(float(lon2))

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        distance_list.append(distance)
        
    top_bike['Distance'] = distance_list
    final_topbike = top_bike[top_bike['Distance'] != 0.000000]
    
    frames = [final_topbikes, final_topbike]
    final_topbikes = pd.concat(frames)

In [ ]:
final_topbikes = final_topbikes.drop_duplicates()

# Mittelere Strecke

In [ ]:
final_topbikes['Distance'].mean()

# Anzahl Bewegungen

In [ ]:
final_topbikes['Distance'].count()

In [ ]:
final_topbikes['Distance'].count() / len(list(set(list(final_topbikes['id'])))) / 14

In [ ]:
final_topbikes['Distance'].count() * 1.5

# Total Kilometer (Luftlinie)

In [ ]:
round(final_topbikes['Distance'].sum())

# Zu welcher Tageszeit werden die Velos am meisten benutzt?

In [ ]:
def pmam(x):
    x = str(x)
    #x = (':'.join(a+b for a,b in zip(x[::2], x[1::2])))
    if x == 'NaN':
        pass
    try:
        x = str(x[:2] + ':' + x[2:])
        date = dateutil.parser.parse(x)
        return str(date.strftime('%d/%m/%Y %H:%M %p'))
    except:
        return 'NaN'

print(pmam('2017-09-09 07:00:03'))

### Adding on two hour, because time on the server is wrong

In [ ]:
def addtwo(elem):
    mytime = datetime.strptime(elem, "%Y-%m-%d %H:%M:%S")
    mytime += timedelta(hours=2)
    return mytime.strftime("%Y.%m.%d %H:%M:%S")

In [ ]:
final_topbikes['Timestamp +2'] = final_topbikes['Timestamp'].apply(addtwo)

In [ ]:
final_topbikes['Timestamp index'] = final_topbikes['Timestamp +2'].apply(pmam)
final_topbikes['Timestamp index'] = final_topbikes['Timestamp index'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%d/%m/%Y %H:%M %p'))
final_topbikes.index = final_topbikes['Timestamp index']

# When are the most bikes rented?

In [ ]:
final_topbikes.groupby(by=final_topbikes.index.hour)['id'].count().plot(kind='bar', figsize=(12,6))

# Median Distance according to time of day

In [ ]:
final_topbikes.groupby(by=final_topbikes.index.hour)['Distance'].median().plot(kind='bar', figsize=(12,6))

# What about day of week?

In [ ]:
final_topbikes.groupby(by=final_topbikes.index.weekday)['id'].count().plot(kind='bar', figsize=(6,3))

In [ ]:
final_topbikes.groupby(by=final_topbikes.index.weekday)['Distance'].count().plot(kind='bar', figsize=(12,6))

In [ ]:
final_topbikes.groupby(by=final_topbikes.index.weekday)['Distance'].median().plot(kind='bar', figsize=(12,6))

In [ ]:
len(list(set(list(final_topbikes['id']))))

# Die Längste Strecken

In [ ]:
final_topbikes.sort_values(by='Distance', ascending=False)

# Saving Top Bikes

In [ ]:
final_topbikes.to_csv('final_top_bikes.csv')

# Looking for Info for Carto

In [ ]:
df.info()

In [ ]:
#Getting right time
del_id = list(df_never_moved['id'])
df['Timestamp +2'] = df['Timestamp'].apply(addtwo)

In [ ]:
#Developing list for the dates
times = df['Timestamp +2'].value_counts().reset_index()
times['date'] = times['index'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%Y.%m.%d %H:%M:%S'))
date_list = list(times.sort_values(by='date')['index'])

In [ ]:
final_date_list = []

for elem in date_list:
    if '07:00' in elem:
        final_date_list.append(elem)
    elif '12:00' in elem:
        final_date_list.append(elem)
    elif '15:00' in elem:
        final_date_list.append(elem)
    elif '00:00' in elem:
        final_date_list.append(elem)
    else:
        pass

In [ ]:
#Pulling out relevant files to make final df for carto
df_carto = df.head(0)
for elem in final_date_list:
    df_new = df[df['Timestamp +2'] == elem]
    df_carto = pd.concat([df_carto, df_new])

In [ ]:
df_carto.head()

In [ ]:
del df['Timestamp']

In [ ]:
df_carto.to_csv('obike_carto.csv')

In [ ]:
len(final_date_list)

In [ ]:
36390 / 55